In [6]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from model import TransformerModel

# Data processing and model compiling

In [2]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [3]:
import io
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter)

def data_process(raw_text_iter):
    data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                       dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [4]:
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

# Training

In [7]:
best_val_loss = float("inf")
epochs = 4 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

c:\users\kajud\appdata\local\programs\python\python38\lib\site-packages\torch\optim\lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 17.93 | loss  8.33 | ppl  4128.56
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 16.83 | loss  6.94 | ppl  1028.08
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 16.79 | loss  6.48 | ppl   650.24
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 16.82 | loss  6.33 | ppl   559.23
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 17.60 | loss  6.20 | ppl   494.23
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 16.96 | loss  6.16 | ppl   474.82
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 16.83 | loss  6.12 | ppl   455.64
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 16.81 | loss  6.11 | ppl   452.41
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 16.82 | loss  6.03 | ppl   417.73
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 16.83 | loss  6.02 | ppl   412.84
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 16.79 | loss  5.90 | ppl   366.62
| epoch   1 |  2400/ 

# Evaluating

In [52]:
data, targets = get_batch(train_data, 2) # 1 szavankent vannak shiftelve a szekvenciak, target = kov input szekvencia

In [53]:
print("Input:")
' '.join([vocab.itos[i] for i in data[:,0].tolist()])

Input:


'chronicles iii = senjō no valkyria 3 <unk> chronicles ( japanese 戦場のヴァルキュリア3 , lit . valkyria of the battlefield 3 ) , commonly referred to as valkyria chronicles iii outside japan , is a tactical'

In [56]:
print("Target:")
' '.join([vocab.itos[i] for i in targets.view(bptt, batch_size)[:,0].tolist()])

Target:


'iii = senjō no valkyria 3 <unk> chronicles ( japanese 戦場のヴァルキュリア3 , lit . valkyria of the battlefield 3 ) , commonly referred to as valkyria chronicles iii outside japan , is a tactical role'

In [91]:
sentence = test_data[35:70,9].unsqueeze(0).cuda()
generated = sentence.transpose(0, 1)

In [92]:
print("Generating text with seed:")
' '.join([vocab.itos[i] for i in generated.transpose(0,1).tolist()[0]])

Generating text with seed:


'basketball player who represented australia in <unk> at the 2012 summer paralympics in london , coming 11th and 12th in her events . <unk> to wheelchair basketball , she made her debut with the national'

In [93]:
model.eval()
softmax = torch.nn.Softmax(dim = 1)
data = generated
for i in range(30):
    src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
    output = model(data, src_mask)
    output_flat = torch.argmax(softmax(output.view(-1, ntokens)), dim = 1)
    generated = torch.cat((generated, output_flat[-1].view(-1,1)), dim = 0)
    data = generated[-bptt:]

In [94]:
print("Generated text:")
' '.join([vocab.itos[i] for i in generated.transpose(0,1).tolist()[0]])

Generated text:


'basketball player who represented australia in <unk> at the 2012 summer paralympics in london , coming 11th and 12th in her events . <unk> to wheelchair basketball , she made her debut with the national hockey league , and was a first time , and was a first time , and was a first time , and a first down , and a first down'